# Wed Scraping(Data Extraction)

In [16]:
# Dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import re
import pymongo
import pandas as pd
from sqlalchemy import create_engine,inspect, func
from config import api_key

In [17]:
# This is to initialize Splinter for Windows users
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [18]:
# list to store extracted values
Link_list=[]
title_list=[]
price_list=[]
loc_list=[]
date_list=[]
# loop to scrap website(for Toronto city) up to 30 pages
for page in range(2,30):
    url = "https://www.kijiji.ca/b-tires-rims/city-of-toronto/winter-tires/page-{}/k0c320l1700273".format(page)
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('div', class_='search-item')
    
    for result in results:
        try:
        # Scrape the link
            url = result.find('div', class_='title').a['href']
            final_url="https://www.kijiji.ca" + url
            Link_list.append(final_url)
    
        # scrape the price
            price = result.find('div', class_='price').text
            price_list.append(price)
        # scrape the  Title
            title = result.find('div', class_='title').text
            title_list.append(title)
            
        # scrape the  Location
            location = result.find('div', class_='location').text
            loc_list.append(location)
        
        # scrape the  Date
            date = result.find('span', class_='date-posted').text
            date_list.append(date)
   
    
        except AttributeError:
            date=''
            
# store the extracted valuse in dictionary            
Dict={}
Dict["Link"]=Link_list
Dict["Title"]=title_list
Dict["Price"]=price_list
Dict["Location"]=loc_list
Dict["Date"]=date_list




# Data Transformation

In [19]:
# Convert dictionary into dataframe
Toronto_df=pd.DataFrame.from_dict(Dict, orient='index')
Toronto_df=Toronto_df.transpose()

In [20]:
# Data cleaning
Toronto_df.Location="Toronto"
Toronto_df["Title"] = Toronto_df["Title"].str.replace("\n", "")
Toronto_df["Price"] = Toronto_df["Price"].str.replace("\n", "")
Toronto_df["Price"] = Toronto_df["Price"].str.replace("$", "")
Toronto_df["Price"] = Toronto_df["Price"].str.replace(",", "")


In [21]:
indexNames = Toronto_df[ Toronto_df['Date'] == "< 2 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 3 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 4 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 8 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 5 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "Yesterday" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 6 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 55 minutes ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 7 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 7 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 9 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 34 minutes ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 35 minutes ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 36 minutes ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 54 minutes ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 59 minutes ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 10 hours ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 60 minutes ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 57 minutes ago" ].index
Toronto_df.drop(indexNames , inplace=True)
indexNames = Toronto_df[ Toronto_df['Date'] == "< 58 minutes ago" ].index
Toronto_df.drop(indexNames , inplace=True)


Toronto_df.head()

,Link,Title,Price,Location,Date
0,https://www.kijiji.ca/v-tires-rims/city-of-tor...,"OEM BMW 17"" rims w...",950.00,Toronto,14/12/2019
1,https://www.kijiji.ca/v-tires-rims/gta-greater...,Burning Rubber Tir...,1300.00,Toronto,14/11/2019
2,https://www.kijiji.ca/v-tires-rims/city-of-tor...,Wanted:Snow tires ...,900.00,Toronto,11/12/2019
14,https://www.kijiji.ca/v-tires-rims/gta-greater...,Burning Rubber Tir...,710.00,Toronto,03/11/2019
15,https://www.kijiji.ca/v-tires-rims/city-of-tor...,225/50/R17 Winter ...,360.00,Toronto,29/11/2019


In [22]:
# Change the datatype of Price and date
Toronto_df = pd.read_excel('temp.xlsx')
Toronto_data=Toronto_df[["New_ID","Link","Title","Price","Location","Date"]]
Toronto_data.set_index("New_ID", inplace=True)
Toronto_data=Toronto_data.astype({"Price":float})
Toronto_data['Date'] = pd.to_datetime(Toronto_data['Date'])

Toronto_data.head()

,Link,Title,Price,Location,Date
New_ID,,,,,
1,https://www.kijiji.ca/v-tires-rims/city-of-tor...,BRAND NEW WINTER T...,100.0,Toronto,2019-02-11
2,https://www.kijiji.ca/v-tires-rims/city-of-tor...,*~*~*~*~HIGH QUALI...,68.0,Toronto,2019-11-20
3,https://www.kijiji.ca/v-tires-rims/city-of-tor...,Winter tires set,150.0,Toronto,2019-11-12
4,https://www.kijiji.ca/v-tires-rims/city-of-tor...,235-70-16 GOODYE...,390.0,Toronto,2019-12-13
5,https://www.kijiji.ca/v-tires-rims/city-of-tor...,Mercedes Benz GLA2...,2400.0,Toronto,2019-07-12


# Extract and Transform for Thunder Bay (web scraping)

In [23]:
# This is to initialize Splinter for Windows users
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [24]:
# list to store extracted values
Link_list=[]
title_list=[]
price_list=[]
loc_list=[]
date_list=[]
# loop to scrap website up to 10 pages
for page in range(2,10):
    url = "https://www.kijiji.ca/b-tires-rims/thunder-bay/winter-tires/page-{}/k0c320l1700126?radius=20.0&address=Thunder+Bay%2C+ON&ll=48.380895,-89.247682".format(page)
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find_all('div', class_='search-item')
    
    for result in results:
        try:
        # Scrape the link
            url = result.find('div', class_='title').a['href']
            final_url="https://www.kijiji.ca" + url
            Link_list.append(final_url)
    
        # scrape the price
            price = result.find('div', class_='price').text
            price_list.append(price)
        # scrape the  Title
            title = result.find('div', class_='title').text
            title_list.append(title)
            
        # scrape the  Location
            location = result.find('div', class_='location').text
            loc_list.append(location)
        
        # scrape the  Date
            date = result.find('span', class_='date-posted').text
            date_list.append(date)
   
    
        except AttributeError:
            date=''
            
# store the extracted valuse in dictionary            
Dict1={}
Dict1["Link"]=Link_list
Dict1["Title"]=title_list
Dict1["Price"]=price_list
Dict1["Location"]=loc_list
Dict1["Date"]=date_list



In [25]:
# Convert dictionary into dataframe
Thunder_df=pd.DataFrame.from_dict(Dict1, orient='index')
Thunder_df=Thunder_df.transpose()
Thunder_df.head()

,Link,Title,Price,Location,Date
0,https://www.kijiji.ca/v-tires-rims/ontario/mer...,\n\n Mercedes Benz ...,"\n $1,094.00\n\n\n\n\n",\n Ontario,15/12/2019
1,https://www.kijiji.ca/v-tires-rims/ontario/201...,\n\n 2019 - 2020 BM...,"\n $1,058.00\n\n\n\n\n",\n Ontario,14/12/2019
2,https://www.kijiji.ca/v-tires-rims/ontario/201...,\n\n 2019 - 2020 BM...,"\n $1,179.00\n\n\n\n\n",\n Ontario,14/12/2019
3,https://www.kijiji.ca/v-tires-rims/thunder-bay...,\n\n Near New - 4x ...,\n $325.00,\n Thunder Bay15/12/2019\n,14/12/2019
4,https://www.kijiji.ca/v-tires-rims/ontario/toy...,\n\n Toyota Camry W...,\n $580.00\n\n\n\n\n,\n Ontario,14/12/2019


In [26]:
# Data cleaning
Thunder_df.Location="Thunder Bay"
Thunder_df["Title"] = Thunder_df["Title"].str.replace("\n", "")
Thunder_df["Price"] = Thunder_df["Price"].str.replace("\n", "")
Thunder_df["Price"] = Thunder_df["Price"].str.replace("$", "")
Thunder_df.count()

Link        176
Title       176
Price       176
Location    176
Date        112
dtype: int64

In [27]:
# Change the datatype of Price and date
Thunder_df = pd.read_excel('temp1.xlsx')
Thunder_df.set_index('New_ID',inplace=True)
Thunder_df=Thunder_df.astype({"Price":float})
Thunder_df['Date'] = pd.to_datetime(Thunder_df['Date'])
Thunder_df.head()

,Link,Title,Price,Location,Date
New_ID,,,,,
1,https://www.kijiji.ca/v-tires-rims/thunder-bay...,17 inch Mazda 3 wi...,450.0,Thunder Bay,2019-12-14
2,https://www.kijiji.ca/v-tires-rims/thunder-bay...,"18"" OEM Toyota Ste...",650.0,Thunder Bay,2019-12-14
3,https://www.kijiji.ca/v-tires-rims/thunder-bay...,Near New - 4x 215/...,325.0,Thunder Bay,2019-12-14
4,https://www.kijiji.ca/v-tires-rims/thunder-bay...,245 65 r17 winter ...,100.0,Thunder Bay,2019-12-14
5,https://www.kijiji.ca/v-tires-rims/thunder-bay...,Open Country Studl...,500.0,Thunder Bay,2019-12-13


# Extract and Transform for Weather API


In [28]:
#
cities = ['Toronto', 'Thunder Bay', 'Windsor', 'Winnipeg']
url = 'https://api.openweathermap.org/data/2.5/weather?'
units = "metric"
query_url = f"{url}appid={api_key}&units={units}&q="
name = []
temp= []
Humidity =[]
Cloudiness=[]
Wind_Speed=[]
weathercondition=[]

record = 1
for city in cities:
    try:   
        response = requests.get(query_url + city).json()
        print(f"Retrieving data for {city}, with city number:{record}")
        name.append(response['name'])
        temp.append(response['main']['temp'])
        Humidity.append(response['main']['humidity'])
        Cloudiness.append(response['clouds']['all'])
        Wind_Speed.append(response['wind']['speed'])
        weathercondition.append(response['weather'][0]['main'])
        record= record + 1
        
        time.sleep(1.01)
    except:
        print("City not found. Skipping...")
    continue

Retrieving data for Toronto, with city number:1
City not found. Skipping...
Retrieving data for Thunder Bay, with city number:2
City not found. Skipping...
Retrieving data for Windsor, with city number:3
City not found. Skipping...
Retrieving data for Winnipeg, with city number:4
City not found. Skipping...


In [29]:
# extracting the data into dictionary
weather_dict = {'location': name,               
                'temp': temp,
                'Humidity': Humidity,
                'Cloudiness': Cloudiness,
                'Wind_Speed': Wind_Speed,
               'weathercondition':weathercondition}

In [30]:
# setting the index to 'New_ID'
weather_df = pd.DataFrame(weather_dict)
weather_df['New_ID'] = df.index + 1
weather_df=weather_df.set_index('New_ID')
weather_df

,location,temp,Humidity,Cloudiness,Wind_Speed,weathercondition
New_ID,,,,,,
2,Toronto,-11.72,51,1,9.3,Clear
3,Thunder Bay,-14.78,59,40,2.1,Clouds
4,Windsor,-10.16,52,75,4.1,Clouds
5,Winnipeg,-13.50,71,75,6.7,Clouds


# Join datasets from Web Scraping and API

In [38]:
# Joining the data frame by common column
merge_data=pd.merge(Thunder_df,weather_df,how='inner',left_on='Location',right_on='location')
merge_data.head()

,Link,Title,Price,Location,Date,location,temp,Humidity,Cloudiness,Wind_Speed,weathercondition
0,https://www.kijiji.ca/v-tires-rims/thunder-bay...,17 inch Mazda 3 wi...,450.0,Thunder Bay,2019-12-14,Thunder Bay,-14.78,59,40,2.1,Clouds
1,https://www.kijiji.ca/v-tires-rims/thunder-bay...,"18"" OEM Toyota Ste...",650.0,Thunder Bay,2019-12-14,Thunder Bay,-14.78,59,40,2.1,Clouds
2,https://www.kijiji.ca/v-tires-rims/thunder-bay...,Near New - 4x 215/...,325.0,Thunder Bay,2019-12-14,Thunder Bay,-14.78,59,40,2.1,Clouds
3,https://www.kijiji.ca/v-tires-rims/thunder-bay...,245 65 r17 winter ...,100.0,Thunder Bay,2019-12-14,Thunder Bay,-14.78,59,40,2.1,Clouds
4,https://www.kijiji.ca/v-tires-rims/thunder-bay...,Open Country Studl...,500.0,Thunder Bay,2019-12-13,Thunder Bay,-14.78,59,40,2.1,Clouds


In [40]:
# Joining the data frame by common column
merge_data1=pd.merge(Toronto_data,weather_df,how='inner',left_on='Location',right_on='location')
merge_data1.head()

,Link,Title,Price,Location,Date,location,temp,Humidity,Cloudiness,Wind_Speed,weathercondition
0,https://www.kijiji.ca/v-tires-rims/city-of-tor...,BRAND NEW WINTER T...,100.0,Toronto,2019-02-11,Toronto,-11.72,51,1,9.3,Clear
1,https://www.kijiji.ca/v-tires-rims/city-of-tor...,*~*~*~*~HIGH QUALI...,68.0,Toronto,2019-11-20,Toronto,-11.72,51,1,9.3,Clear
2,https://www.kijiji.ca/v-tires-rims/city-of-tor...,Winter tires set,150.0,Toronto,2019-11-12,Toronto,-11.72,51,1,9.3,Clear
3,https://www.kijiji.ca/v-tires-rims/city-of-tor...,235-70-16 GOODYE...,390.0,Toronto,2019-12-13,Toronto,-11.72,51,1,9.3,Clear
4,https://www.kijiji.ca/v-tires-rims/city-of-tor...,Mercedes Benz GLA2...,2400.0,Toronto,2019-07-12,Toronto,-11.72,51,1,9.3,Clear


# Create database connection 

In [31]:
connection_string = "postgres:Ruhi@localhost:5432/ETLDb"
engine = create_engine(f'postgresql://{connection_string}')

In [32]:
# Confirm tables
#engine.table_names()
inspector = inspect(engine)
inspector.get_table_names()

['ThunderBay', 'toronto', 'weatherdata', 'Toronto']

# Load DataFrames into database

In [41]:
Toronto_data.to_sql(name='Toronto', con=engine, if_exists='append', index=True)

In [34]:
Thunder_df.to_sql(name='ThunderBay', con=engine, if_exists='append', index=True)

In [35]:
weather_df.to_sql(name='weatherData', con=engine, if_exists='append', index=True)